In [5]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

def prepare_data():
    # Charger le dataset Iris
    iris = datasets.load_iris()
    X = iris.data[:, :2]  # Utiliser uniquement les deux premières caractéristiques pour simplification
    y = iris.target

    # Binariser le dataset: 1 si la classe n'est pas 0, sinon 0
    y = (y != 0).astype(int)

    # Séparer les données en ensembles de formation et de test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Retourner les ensembles
    return X_train, X_test, y_train, y_test

# Exécuter la fonction et afficher les résultats
X_train, X_test, y_train, y_test = prepare_data()
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (105, 2)
y_train shape: (105,)
X_test shape: (45, 2)
y_test shape: (45,)


In [6]:
import dash
import numpy as np
import plotly.graph_objs as go
from dash import dcc, html
from dash.dependencies import Input, Output
from sklearn.ensemble import IsolationForest
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold
from skopt import BayesSearchCV
from skopt.space import Integer, Real, Categorical

# Données simulées pour l'exemple, remplacez par vos vraies données
X_train = np.random.rand(100, 4)
y_train = (np.random.rand(100) > 0.5).astype(int)

# Définir un scorer personnalisé
def custom_scorer(y_true, y_pred):
    return np.mean(y_true == y_pred)

scorer = make_scorer(custom_scorer, greater_is_better=True)

# Initialiser l'application Dash
app = dash.Dash(__name__)

# Mémoire partagée pour stocker les résultats d'optimisation
results = {
    'time': [],
    'score': []
}

# Définir la mise en page de l'application
app.layout = html.Div([
    dcc.Graph(id='live-update-graph'),
    dcc.Interval(id='interval-component', interval=1*1000, n_intervals=0)
])

# Callback pour la mise à jour en temps réel
@app.callback(Output('live-update-graph', 'figure'),
              Input('interval-component', 'n_intervals'))
def update_graph_live(n_intervals):
    trace = go.Scatter(
        x=results['time'],
        y=results['score'],
        name='Optimization Performance',
        mode='lines+markers'
    )

    return {
        'data': [trace],
        'layout': go.Layout(title='Real-time Performance Over Time')
    }

# Fonction pour lancer l'optimisation Bayésienne
def run_optimization():
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    search_space = {
        'n_estimators': Integer(100, 300),
        'max_samples': Integer(50, 200),
        'contamination': Real(0.01, 0.05),
        'max_features': Real(0.5, 1.0),
        'bootstrap': Categorical([True, False])
    }

    opt = BayesSearchCV(
        estimator=IsolationForest(random_state=42),
        search_spaces=search_space,
        n_iter=10,
        scoring=scorer,
        cv=kf,
        n_jobs=-1
    )

    for i, result in enumerate(opt.fit(X_train, y_train)):
        results['time'].append(i)
        results['score'].append(-result.fun)

# Exécuter l'optimisation dans un thread séparé pour ne pas bloquer Dash
from threading import Thread
thread = Thread(target=run_optimization)
thread.start()

# Exécuter l'application Dash
if __name__ == '__main__':
    app.run_server(debug=True)


c:\Users\decroux paul\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_iforest.py:307: UserWarning:

max_samples (130) is greater than the total number of samples (100). max_samples will be set to n_samples for estimation.

Exception in thread Thread-5 (run_optimization):
Traceback (most recent call last):
  File "c:\Users\decroux paul\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\decroux paul\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\decroux paul\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\decroux paul\AppData\Local\Temp\ipykernel_25128\1792223504.py", line 73, in run_optimization
TypeError: 'BayesSearchCV' object is not iterable
